In [51]:
from math import radians, cos, sin, asin, sqrt
from sklearn import preprocessing
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

#test set from data driven

In [52]:
import pandas as pd
df_train_label=pd.read_csv('/content/train labels.csv')
df_train_var=pd.read_csv('/content/train variables.csv')
df=df_train_var.merge(df_train_label)

In [53]:
import pandas as pd
df_test=pd.read_csv('/content/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')
df_test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [54]:
df.gps_height=df.gps_height.replace(0,df.gps_height.median())
#df['funder']=df['funder'].replace(np.nan,df['funder'].mode)
df['funder'] = df['funder'].fillna('unknown')
df['installer']=df['installer'].fillna('unknown')
df['subvillage']=df['subvillage'].fillna('unknown')
df['public_meeting']=df['public_meeting'].fillna('unknown')
df['scheme_management']=df['scheme_management'].fillna('unknown')
#df.amount_tsh=df.amount_tsh.replace(0,df.amount_tsh.median())

In [55]:
df.isnull().values.any()

True

In [56]:
z=sum(pd.isnull(df.installer))
print('number of null values',z)

#df.isnull().sum()

number of null values 0


In [57]:
df.public_meeting.replace([True, False,'NaN'],[1,0,1]) 

0              1
1        unknown
2              1
3              1
4              1
          ...   
59395          1
59396          1
59397          1
59398          1
59399          1
Name: public_meeting, Length: 59400, dtype: object

In [58]:
#dropped features
#recordeed_by drop has constant value,scheme_name has 47% null values drop,
#permit drop public meeting,
#payment_type,quality group,quantity_group,source,source_class,waterpoint_type

In [59]:
#https://datascience.stackexchange.com/questions/49553/combining-latitude-longitude-position-into-single-feature?rq=1

def single_pt_haversine(lat, lng, degrees=True):
    """
    'Single-point' Haversine: Calculates the great circle distance
    between a point on Earth and the (0, 0) lat-long coordinate
    """
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if degrees:
        lat, lng = map(radians, [lat, lng])

    # 'Single-point' Haversine formula
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    d = 2 * r * asin(sqrt(a)) 

    return d

df['harvesine_distance'] = [single_pt_haversine(x, y) for x, y in zip(df.latitude, df.longitude)]

In [60]:
df_test['harvesine_distance'] = [single_pt_haversine(x, y) for x, y in zip(df_test.latitude, df_test.longitude)]


In [61]:
df['public_meeting']=df['public_meeting'].fillna(df['public_meeting'].mode().values[0])

In [62]:
z=sum(pd.isnull(df.public_meeting))
print('number of null values',z)

number of null values 0


In [63]:
def prepare_data(df):
  X=df.drop(['status_group'],axis=1)
  #df.gps_height=df.gps_height.replace(0,df.gps_height.median())
  df.gps_height=df.gps_height.where(df.gps_height< 0, df.gps_height.mean())
  df.population=df.population.replace(0,df.population.mean())
  #df['funder']=df['funder'].replace(np.nan,df['funder'].mode)
  X['funder'] = X['funder'].fillna('unknown')
  X['installer']=X['installer'].fillna('unknown')
  X['subvillage']=X['subvillage'].fillna('unknown')
  X['public_meeting']=X['public_meeting'].fillna('unknown')
  X['scheme_management']=X['scheme_management'].fillna('unknown')
  df.amount_tsh=df.amount_tsh.replace(0,df.amount_tsh.median())
  y=df.status_group
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.05,random_state=42)
  from sklearn import preprocessing
  le = preprocessing.LabelEncoder()
  le=le.fit(df.status_group)
  #list(le.classes_)
  y_train=le.transform(y_train)
  y_test=le.transform(y_test)
  

  # We are considering only the words which appeared in at least 10 documents(rows or projects).
  def dummy_enc(df,feature):
    #df[feature] = df[feature].fillna("Unknown")
    vectorizer = CountVectorizer(min_df=5)
    train = vectorizer.fit_transform(X_train[feature])
    test = vectorizer.transform(X_test[feature])
    return train,test
  
  #X_train['funder'] = X_train['funder'].fillna("Unknown")
  #X_test['funder'] = X_test['funder'].fillna("Unknown")
  funder_ohe_train,funder_ohe_test=dummy_enc(X_train,'funder')
 
  #X_train['installer'],X_test['installer'] = X_train['installer'].fillna("Unknown"),X_test['installer'].fillna("Unknown")
  installer_ohe_train,installer_ohe_test=dummy_enc(X_train,'installer')
  
  #
  wpt_name_ohe_train,wpt_name_ohe_test=dummy_enc(X_train,'wpt_name')

  basin_dummy_ohe_train,basin_dummy_ohe_test = dummy_enc(X_train,'basin') 
  
  subvillage_ohe_train,subvillage_ohe_test = dummy_enc(X_train,'subvillage')

  region_ohe_train,region_ohe_test = dummy_enc(X_train,'region') 

  lga_ohe_train,lga_ohe_test = dummy_enc(X_train,'lga')

  ward_ohe_train,ward_ohe_test = dummy_enc(X_train,'ward')
 
  #public_meeting_ohe_train,public_meeting_ohe_test = dummy_enc(X_train,'public_meeting')

  scheme_management_ohe_train,scheme_management_ohe_test=dummy_enc(X_train,'scheme_management')

  #scheme_name_ohe_train,scheme_name_ohe_test = dummy_enc(X_train,'scheme_name') #47% null values

  extraction_type_ohe_train,extraction_type_ohe_test = dummy_enc(X_train,'extraction_type')  

  management_group_ohe_train,management_group_ohe_test = dummy_enc(X_train,'management_group')
  payment_type_ohe_train,payment_type_ohe_test = dummy_enc(X_train,'payment_type')

  water_quality_ohe_train,water_quality_ohe_test = dummy_enc(X_train,'water_quality')

  quantity_ohe_train,quantity_ohe_test = dummy_enc(X_train,'quantity')

  source_ohe_train,source_ohe_test = dummy_enc(X_train,'source')

  source_class_ohe_train,source_class_ohe_test= dummy_enc(X_train,'source_class')

  waterpoint_type_ohe_train,waterpoint_type_ohe_test = dummy_enc(X_train,'waterpoint_type')

  #numerical features

  df.gps_height=df.gps_height.replace(0,df.construction_year.median())
  gps_height_train=X_train['gps_height'].values.reshape(-1,1)
  gps_height_test=X_test['gps_height'].values.reshape(-1,1)
  gps_height_train = preprocessing.StandardScaler().fit_transform(gps_height_train)
  gps_height_test  = preprocessing.StandardScaler().fit_transform(gps_height_test)

  amount_tsh_train=X_train['amount_tsh'].values.reshape(-1,1)
  amount_tsh_test=X_test['amount_tsh'].values.reshape(-1,1)
  amount_tsh_train = preprocessing.StandardScaler().fit_transform(amount_tsh_train)
  amount_tsh_test  = preprocessing.StandardScaler().fit_transform(amount_tsh_test)

  harvesine_distance_train=X_train['harvesine_distance'].values.reshape(-1,1)
  harvesine_distance_test=X_test['harvesine_distance'].values.reshape(-1,1)
  harvesine_distance_train = preprocessing.StandardScaler().fit_transform(harvesine_distance_train)
  harvesine_distance_test  = preprocessing.StandardScaler().fit_transform(harvesine_distance_test)

  region_code_train=X_train['region_code'].values.reshape(-1,1)
  region_code_test=X_test['region_code'].values.reshape(-1,1)
  region_code_train = preprocessing.StandardScaler().fit_transform(region_code_train)
  region_code_test  = preprocessing.StandardScaler().fit_transform(region_code_test)
  region_code_train.shape,region_code_test.shape

  population_train=X_train['population'].values.reshape(-1,1)
  population_test=X_test['population'].values.reshape(-1,1)
  population_train = preprocessing.StandardScaler().fit_transform(population_train)
  population_test  = preprocessing.StandardScaler().fit_transform(population_test)
  population_train.shape,population_test.shape



  train=hstack((funder_ohe_train, installer_ohe_train,wpt_name_ohe_train,basin_dummy_ohe_train,
              subvillage_ohe_train,region_ohe_train,lga_ohe_train,ward_ohe_train,
              scheme_management_ohe_train,extraction_type_ohe_train,
            management_group_ohe_train,payment_type_ohe_train,water_quality_ohe_train,
            water_quality_ohe_train,quantity_ohe_train,source_ohe_train,
            source_class_ohe_train,source_class_ohe_train,waterpoint_type_ohe_train,
            gps_height_train,amount_tsh_train,harvesine_distance_train,
            population_train)).tocsr() 

  test=hstack((funder_ohe_test, installer_ohe_test,wpt_name_ohe_test,basin_dummy_ohe_test,
              subvillage_ohe_test,region_ohe_test,lga_ohe_test,ward_ohe_test,
              scheme_management_ohe_test,extraction_type_ohe_test,
            management_group_ohe_test,payment_type_ohe_test,water_quality_ohe_test,
            water_quality_ohe_test,quantity_ohe_test,source_ohe_test,
            source_class_ohe_test,source_class_ohe_test,waterpoint_type_ohe_test,
            gps_height_test,amount_tsh_test,harvesine_distance_test,
            population_test)).tocsr()           
  
  return train,test,y_train,y_test       

In [64]:
from scipy.sparse import hstack


In [65]:
train,test,y_train,y_test=prepare_data(df)

In [66]:
#le.inverse_transform([2])#0-functional,1-functional needs repair,non functional


In [67]:
...
# define the multinomial logistic regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class='multinomial')
model.fit(train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(multi_class='multinomial')

In [68]:
classifier=model

In [69]:

### Create a Pickle file using serialization 
import pickle
pickle_out = open("classifier.pkl","wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()

In [70]:
model.predict(test[688])

array([0])

In [71]:
classifier=model

In [72]:
train[0].todense()

matrix([[ 0.        ,  0.        ,  0.        , ..., -0.1054474 ,
          0.22271645, -0.12588015]])